In [ ]:
# sau khi cai xong thi restart lại runtime và chạy từ cell thứ 2
!pip3 install tensorflow==2.2
!pip3 install keras==2.3.1
!pip3 install -U segmentation-models

     |████████████████████████████████| 516.2 MB 4.0 kB/s 
     |████████████████████████████████| 454 kB 51.4 MB/s 
     |████████████████████████████████| 3.0 MB 41.9 MB/s 
     |████████████████████████████████| 2.9 MB 40.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

from segmentation_models.metrics import iou_score
from segmentation_models import Unet
import segmentation_models as sm

sm.set_framework("tf.keras")
sm.framework()

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


'tf.keras'

In [ ]:
# Dataset path
data_train_path  = "/content/drive/MyDrive/nt-dataset/coco-format/train"
data_test_path  = "/content/drive/MyDrive/nt-dataset/coco-format/test"
data_val_path  = "/content/drive/MyDrive/nt-dataset/coco-format/val"

# Kích thước ảnh và số lượng batch
w, h = 512, 512
batch_size = 15

# Dataset va Dataloader
BACKBONE = "resnet34"
preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:
# Tạo bộ dữ liệu load theo batch
class Dataset:
    def __init__(self, image_path, mask_path, w, h):
        # the paths of images
        self.image_path = image_path
        # the paths of segmentation images
        self.mask_path = mask_path

        self.w = w
        self.h = h

    def __getitem__(self, i):
        # read data
        image = cv2.imread(self.image_path[i])
        image = cv2.resize(image, (self.w, self.h), interpolation=cv2.INTER_AREA)
        image = preprocess_input(image)

        mask = cv2.imread(self.mask_path[i], cv2.IMREAD_UNCHANGED)
        image_mask = cv2.resize(mask, (self.w, self.h), interpolation=cv2.INTER_AREA)

        image_mask = [(image_mask == v) for v in [1]]
        image_mask = np.stack(image_mask, axis=-1).astype('float')

        return image, image_mask

In [ ]:
class Dataloader(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size,shape, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.shape = shape
        self.indexes = np.arange(self.shape)

    def __getitem__(self, i):
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.stack(samples, axis=0) for samples in zip(*data)]

        return tuple(batch)

    def __len__(self):
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)


In [ ]:
#  Lấy ảnh từ dataset image_path, mask_path
def load_path(data_train_path, data_test_path):
  img_train_path = []
  mask_train_path = []

  img_test_path = []
  mask_test_path = []

  # train
  img_folder = os.path.join(data_train_path, 'images')
  for file in os.listdir(img_folder):
    img_train_path.append(os.path.join(img_folder, file))

  mask_folder = os.path.join(data_train_path, 'masks')
  for file in os.listdir(mask_folder):
    mask_train_path.append(os.path.join(mask_folder, file))

  # test
  
  img_folder = os.path.join(data_test_path, 'images')
  for file in os.listdir(img_folder):
    img_test_path.append(os.path.join(img_folder, file))

  mask_folder = os.path.join(data_test_path, 'masks')
  for file in os.listdir(mask_folder):
    mask_test_path.append(os.path.join(mask_folder, file))


  return img_train_path, img_test_path, mask_train_path, mask_test_path

In [ ]:
# Load
image_train, image_test, mask_train, mask_test = load_path(data_train_path, data_test_path)

In [ ]:
# Tạo dataset và dataloader
train_dataset = Dataset(image_train, mask_train, w, h)
test_dataset = Dataset(image_test, mask_test, w, h)

train_loader = Dataloader(train_dataset, batch_size, shape=len(image_train), shuffle=True)
test_loader = Dataloader(test_dataset, batch_size, shape=len(image_test), shuffle=True)

In [ ]:
# Khởi tạo model
opt = tf.keras.optimizers.Adam(0.05)
model = Unet(BACKBONE,encoder_weights="imagenet",activation="sigmoid",input_shape=(512, 512, 3),encoder_freeze=True)
loss1 = sm.losses.categorical_focal_dice_loss
model.compile(optimizer=opt,loss=loss1,metrics=[iou_score])

85524480/85521592 [==============================] - 1s 0us/step


In [ ]:
# training
filepath="nuchal-translucency-unet-model.hdf5"
callback = ModelCheckpoint(filepath, monitor='val_iou_score', verbose=1, save_best_only=True, mode='max')
model.fit(train_loader, validation_data=test_loader, epochs=50, callbacks=[callback])

Epoch 1/50
33/33 [==============================] - ETA: 0s - loss: 0.7550 - iou_score: 0.2421  

InvalidArgumentError: ignored

In [ ]:
# Load model de test
model.load_weights(filepath)

ids = range(len(image_test))
index = random.sample(ids, 10)

import matplotlib.pyplot as plt

for id in index:
  # Anh dau vao, ko phai mask
  image = cv2.imread(image_test[id])
  image = cv2.resize(image, (512, 512))
  # Dua qua model de predicted segmentation map

  mask_predict = model.predict(image[np.newaxis, :, :, :])

  # Doc mask thuc te
  image_mask = cv2.imread(mask_test[id], cv2.IMREAD_UNCHANGED)
  image_mask = cv2.resize(image_mask, (512, 512))

  plt.figure(figsize=(10, 6))
  plt.subplot(131)
  plt.title("Hình ảnh sản phẩm")
  plt.imshow(image)
  plt.subplot(132)
  plt.title("Vết lỗi thật")
  plt.imshow(image_mask)
  plt.subplot(133)
  plt.title("Vết lỗi dự đoán")
  z = mask_predict[0]#[:, :, 0]
  plt.imshow(z)
  plt.show()